In [1]:
# spacex_launch_dash = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
# spacex_dash_app = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

In [2]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash # I'm using the anaconda distribution of jupyter for this lab
import plotly.express as px

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.drop(columns=['Unnamed: 0'],inplace=True)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
spacex_df.head().T

,0,1,2,3,4
Flight Number,1,2,3,4,5
Launch Site,CCAFS LC-40,CCAFS LC-40,CCAFS LC-40,CCAFS LC-40,CCAFS LC-40
class,0,0,0,0,0
Payload Mass (kg),0.0,0.0,525.0,500.0,677.0
Booster Version,F9 v1.0 B0003,F9 v1.0 B0004,F9 v1.0 B0005,F9 v1.0 B0006,F9 v1.0 B0007
Booster Version Category,v1.0,v1.0,v1.0,v1.0,v1.0


In [5]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [6]:
spacex_df.groupby('Launch Site')['class'].mean()

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

In [7]:
print(min_payload)
print(max_payload)

0.0
9600.0


In [ ]:
# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div(dcc.Dropdown(id='site-dropdown',
                                                      options=[
                                                          {'label':'All Sites','value':'ALL'},
                                                          {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                                          {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                                          {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                          {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}],
                                                      value='ALL', 
                                                      placeholder='Select a Launch Site here', 
                                                      searchable = True)),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                         min=0, 
                                                         max=10000, 
                                                         step=1000, 
                                                         marks={0: '0', 
                                                                2500: '2.500', 
                                                                5000: '5.000',
                                                                7500: '7.500',
                                                                10000: '10.000'},
                                                         value=[min_payload, max_payload])),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df
        fig = px.pie(filtered_df, values='class', 
                     names = 'Launch Site', 
                     title = 'Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        success = len(filtered_df[filtered_df['class']==1])
        failure = len(filtered_df[filtered_df['class']==0])
        fig = px.pie(filtered_df, values=[success,failure], 
                     names = ['Success','Failure'], 
                     title = f'Total Success Launches for Site {entered_site}')
        return fig
        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site,payload_slider):
    low, high = payload_slider 
    if entered_site == 'ALL':
        filtered_df = spacex_df
        mask = (filtered_df['Payload Mass (kg)'] > low) & (filtered_df['Payload Mass (kg)'] < high)
        fig = px.scatter(filtered_df[mask], 
                         x = 'Payload Mass (kg)', 
                         y = 'class',
                         color = 'Booster Version Category', 
                         title = 'Correlation between Payload and Success for All Sites')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        mask = (filtered_df['Payload Mass (kg)'] > low) & (filtered_df['Payload Mass (kg)'] < high)
        fig = px.scatter(filtered_df[mask], 
                         x = 'Payload Mass (kg)', 
                         y = 'class',
                         color = 'Booster Version Category',
                         title = f'Correlation between Payload and Success for Site {entered_site}')
        return fig
    
# Run the app
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)